# Лабораторная работа No 9
## Алгоритмы кластерного анализа данных

In [2]:
import pandas as pd

# pd.set_option('display.max_rows', None)

Исследуем полученные данные

In [23]:
df = pd.read_csv('colleges.csv')
df = df[['okrug', 'X', 'Y']]
df

,okrug,X,Y
0,Юго-Восточный административный округ,"37,850588","55,685852"
1,Южный,"37,657122","55,713902"
2,Центральный административный округ,"37,625878","55,73442"
3,Южный административный округ,"37,654086","55,726615"
4,Южный административный округ,"37,695225","55,688223"
...,...,...,...
280,Центральный административный округ,"37,68165","55,74362"
281,Центральный административный округ,"37,684059","55,736391"
282,Центральный административный округ,"37,684201","55,736755"
283,Восточный административный округ,"37,816595","55,789968"


Координаты не заданы при помощи float-числа. Приведем типы:

In [25]:
df[['X', 'Y']] = df[['X', 'Y']].apply(lambda x: x.str.replace(',', '.', 1)).apply(pd.to_numeric)
df

,okrug,X,Y
0,Юго-Восточный административный округ,37.850588,55.685852
1,Южный,37.657122,55.713902
2,Центральный административный округ,37.625878,55.734420
3,Южный административный округ,37.654086,55.726615
4,Южный административный округ,37.695225,55.688223
...,...,...,...
280,Центральный административный округ,37.681650,55.743620
281,Центральный административный округ,37.684059,55.736391
282,Центральный административный округ,37.684201,55.736755
283,Восточный административный округ,37.816595,55.789968


Найдем округа, зайдествованные в выборке

In [4]:
set(df.okrug)

{'Восточный административный округ',
 'Западный административный округ',
 'Зеленоградский административный округ',
 'Северный административный округ',
 'Северо-Восточный административный округ',
 'Северо-Западный административный округ',
 'Центральный административный округ',
 'Юго-Восточный административный округ',
 'Юго-Западный административный округ',
 'Южный',
 'Южный административный округ'}

У некоторых записей есть ошибка в метке округа - исправим.

In [5]:
df.loc[df.okrug == 'Южный', 'okrug'] = 'Южный административный округ'

In [6]:
set(df.okrug)

{'Восточный административный округ',
 'Западный административный округ',
 'Зеленоградский административный округ',
 'Северный административный округ',
 'Северо-Восточный административный округ',
 'Северо-Западный административный округ',
 'Центральный административный округ',
 'Юго-Восточный административный округ',
 'Юго-Западный административный округ',
 'Южный административный округ'}

Ошибка исправлена

Найдем и подготовим данные центров округов

In [7]:
# Данные по округам с википедии и гугл-карт (скорее всего являются центром массы, но не фактическим центром)
data = [['Восточный административный округ', 'ВАО', 55.7752, 37.9056],
        ['Западный административный округ', 'ЗАО', 55.6756, 37.2327],
        ['Зеленоградский административный округ', 'ЗелАО', 55.9791, 37.2001],
        ['Северный административный округ', 'САО', 55.8587345, 37.3223247],
        ['Северо-Восточный административный округ', 'СВАО', 55.8562, 37.6138],
        ['Северо-Западный административный округ', 'СЗАО', 55.8131, 37.4201],
        ['Центральный административный округ', 'ЦАО', 55.744, 37.6165],
        ['Юго-Восточный административный округ', 'ЮВАО', 55.6828, 37.8053],
        ['Юго-Западный административный округ', 'ЮЗАО', 55.5756, 37.5444],
        ['Южный административный округ', 'ЮАО', 55.6328, 37.6749]]

reference = pd.DataFrame(data, columns=['full', 'short', 'X', 'Y'])
reference

,full,short,X,Y
0,Восточный административный округ,ВАО,55.775200,37.905600
1,Западный административный округ,ЗАО,55.675600,37.232700
2,Зеленоградский административный округ,ЗелАО,55.979100,37.200100
3,Северный административный округ,САО,55.858734,37.322325
4,Северо-Восточный административный округ,СВАО,55.856200,37.613800
5,Северо-Западный административный округ,СЗАО,55.813100,37.420100
6,Центральный административный округ,ЦАО,55.744000,37.616500
7,Юго-Восточный административный округ,ЮВАО,55.682800,37.805300
8,Юго-Западный административный округ,ЮЗАО,55.575600,37.544400
9,Южный административный округ,ЮАО,55.632800,37.674900
